In [ ]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from tensorflow.keras.layers import Dense, Input, Reshape, Conv2DTranspose, Conv2D, Flatten
from tensorflow.keras.models import Model, Sequential
from moviepy.editor import VideoClip, TextClip
import numpy as np

In [ ]:
class TextToVideoGenerator:
    def __init__(self, gpt_model_path="gpt2", latent_dim=100):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_path)
        self.gpt_model = TFGPT2LMHeadModel.from_pretrained(gpt_model_path)
        self.gan = GAN(latent_dim=latent_dim)

    def text_to_embedding(self, text):
        inputs = self.tokenizer(text, return_tensors="tf", max_length=1024, truncation=True)
        outputs = self.gpt_model(inputs["input_ids"])
        return outputs.last_hidden_state[:, -1, :]

    def generate_video(self, quotes, output_path='output.mp4', duration_per_quote=5, fps=24):
        frames = []
        for quote in quotes:
            embedding = self.text_to_embedding(quote)
            latent_noise = np.random.normal(size=(1, self.gan.latent_dim))
            generated_video = self.gan.generator.predict(latent_noise)

            frame_clip = VideoClip(lambda t: generated_video, duration=duration_per_quote)
            text_clip = TextClip(quote, fontsize=24, color='white', bg_color='black').set_pos('center').set_duration(duration_per_quote)

            final_clip = VideoClip(composite_video_clip, size=(generated_video.shape[1], generated_video.shape[2]))
            frames.append(final_clip)

        final_video = concatenate_videoclips(frames, method="compose")
        final_video.write_videofile(output_path, codec='libx264', audio_codec='aac', fps=fps)

In [ ]:
class GAN:
    def __init__(self, latent_dim=100, img_shape=(64, 64, 3)):
        self.latent_dim = latent_dim
        self.img_shape = img_shape
        self.generator = self.build_generator()
        self.discriminator = self.build_discriminator()
        self.gan = self.build_gan()

    def build_generator(self):
        model = Sequential([
            Dense(128, input_dim=self.latent_dim),
            Reshape((1, 1, 128)),
            Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same", activation="relu"),
            Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same", activation="relu"),
            Conv2DTranspose(3, (4, 4), strides=(2, 2), padding="same", activation="sigmoid"),
        ])
        return model

    def build_discriminator(self):
        model = Sequential([
            Conv2D(64, (4, 4), strides=(2, 2), padding="same", input_shape=self.img_shape, activation="relu"),
            Conv2D(128, (4, 4), strides=(2, 2), padding="same", activation="relu"),
            Flatten(),
            Dense(1, activation="sigmoid"),
        ])
        model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
        return model

    def build_gan(self):
        self.discriminator.trainable = False
        gan_input = Input(shape=(self.latent_dim,))
        x = self.generator(gan_input)
        gan_output = self.discriminator(x)
        model = Model(gan_input, gan_output)
        model.compile(loss="binary_crossentropy", optimizer="adam")
        return model

In [ ]:
generator = TextToVideoGenerator()

motivational_quotes = [
    "Believe in yourself and all that you are.",
    "You are stronger than you think.",
    "The only way to do great work is to love what you do.",
    "Don't watch the clock; do what it does. Keep going.",
]

generator.generate_video(motivational_quotes)